<a href="https://colab.research.google.com/github/JoeOlang/NLP/blob/main/Text%20Classification/Swahili/ZindiNews/v1.3_LinearSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import re, string
import nltk
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV


In [3]:
# Introduce some common Swahili stop words
stop_words_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/swahili_stopwords.csv')
stop_words = stop_words_df['StopWords'].tolist()

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Swahili/Train.csv')
df.head()

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa


In [5]:
df['category'].unique()

array(['Kitaifa', 'Biashara', 'michezo', 'Kimataifa', 'Burudani'],
      dtype=object)

---

In [7]:
labels = df['category']
text = df['content']

X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state=0, test_size=0.3)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)

X_test_counts = count_vect.transform(X_test)
X_test_transformed = tf_transformer.transform(X_test_counts)

labels = LabelEncoder()
y_train_labels_fit = labels.fit(y_train)
y_train_lables_trf = labels.transform(y_train)

print(labels.classes_)

['Biashara' 'Burudani' 'Kimataifa' 'Kitaifa' 'michezo']


In [8]:
linear_svc = LinearSVC()
clf = linear_svc.fit(X_train_transformed,y_train_lables_trf)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,
                                        cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train_lables_trf)
predicted = calibrated_svc.predict(X_test_transformed)

In [11]:
to_predict = ["KOCHA wa Senegal, Alliou Cisse amesema timu yake itazoea mechi hadi mechi kwenye michuano ya Afcon akiwa na lengo la kufanya vizuri kuliko michuano ya mwisho iliyofanyika Gabon mwaka 2017, ambapo Senegal iliishia robo fainali.Cisse ni miongoni mwa makocha wachache wanaoonekana kwenye michuano hiyo mfululizo kuanzia ile iliyopita. Simba wa Teranga wamepangwa kundi C pamoja na Algeria, Kenya na Tanzania.Akizungumzia nafasi yake kwenye kundi hilo, Cisse alisema kundi liko sawa. “Ni kundi lililo sawa kwa timu yangu, kwa sasa hakuna timu ndogo au kubwa. Timu zote zilizofuzu zitakuwa Misri kupambana na kujaribu kupata matokeo mazuri.” “Tutakuwa na Algeria sio ngeni kwenye michuano hii, pia kuna Kenya na Tanzania. Hatuzidharau Kenya na Tanzania kutokana na hali zao, tunatakiwa kujiandaa vizuri kushindana.”Mechi yake na Algeria itakuwa ya tatu mfululizo, amejiandaa vipi? “ Ni mara ya tatu tunacheza na Algeria kwenye hatua ya makundi. Tulicheza 2015 (Equatorial Guinea), 2017 (Gabon), na sasa Misri, ni historia ndefu kwa timu hizi Algeria imebadilika tangu kuwasili kwa Djamel Belmadi, ambaye ni rafiki yangu, tumekuwa pamoja majirani Paris itakuwa mechi nzuri.”Senegal na Algeria ndio zinapewa nafasi kubwa kwenye kundi hili, lakini Kenya na Tanzania hazipaswi kudharauliwa. “Naweza kusema kwamba Senegal haiiogopi timu yoyote.” Akizungumzia malengo ya timu yake alisema wataichukulia kila mechi kwa uzito wake, kwa sababu wanajua lolote linaweza kutokea. “Ni kweli ni miongoni mwa tunaopewa nafasi kubwa, lakini kufikia kiwango hicho tunatakiwa kumfunga kila tunayekutana naye, na itaanza na Tanzania Juni 23, tunapaswa kufikiria uwezo wetu na si kuwafikia adui zetu."]
p_count = count_vect.transform(to_predict)
p_tfidf = tf_transformer.transform(p_count)
print('Average accuracy on test set={}'.format(np.mean(predicted == labels.transform(y_test))))
print('Predicted probabilities of demo input string are')
print(calibrated_svc.predict_proba(p_tfidf))

Average accuracy on test set=0.8635187580853816
Predicted probabilities of demo input string are
[[4.72071015e-07 4.78727568e-04 1.25124447e-04 1.81041569e-04
  9.99214634e-01]]


In [63]:
def predict(content):
    p_count = count_vect.transform(content)
    p_tfidf = tf_transformer.transform(p_count)
    predictions = calibrated_svc.predict_proba(p_tfidf)
    predictions = predictions.flatten()
    scores = predictions.tolist()

    return scores

In [64]:
predict(to_predict)

[4.7207101549240433e-07,
 0.00047872756823401065,
 0.0001251244468672724,
 0.00018104156869992066,
 0.9992146343451833]

In [35]:
min(output)

4.7207101549240433e-07

In [56]:
type(arr)

numpy.ndarray